# Data Cleaning

Goal: create a clean, analysis-ready dataset while keeping the raw data unchanged.

Cleaning objectives:
- Convert Order_Date to datetime
- Validate numeric fields (Quantity, Price, Discount, Revenue)
- Check duplicates and logical rules
- Create and export a clean dataset for the next milestone


## Load raw data

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('globex_retail_raw.csv')
df.head()

,Customer_ID,Order_ID,Order_Date,Product_Category,Product_Sub_Category,Quantity,Price,Discount,Customer_Location,Revenue
0,CUST_013738,ORD_00102406,01/01/2023,Home & Garden,Gardening Tools,1,419.19,0.00,TN,419.1900
1,CUST_011726,ORD_00102902,01/01/2023,Electronics,Laptops,1,222.37,0.09,TN,202.3567
2,CUST_010891,ORD_00103864,01/01/2023,Electronics,Laptops,6,1107.65,0.00,IN,6645.9000
3,CUST_011452,ORD_00103560,01/01/2023,Electronics,Gaming Consoles,5,288.84,0.00,MA,1444.2000
4,CUST_010886,ORD_00100632,02/01/2023,Electronics,Headphones,1,191.27,0.00,AZ,191.2700


## Convert Order_Date
Dates currently look like 02/02/2023 (day/month/year), we convert day first dayfirst=True

In [4]:
df["Order_Date"] = pd.to_datetime(df["Order_Date"], dayfirst=True, errors="coerce")
df["Order_Date"].isna().sum() # check for any conversion errors
#df.info()


np.int64(0)

## Validate Discount range

In [5]:
df['Discount'].describe()
# We expect discounts between 0 and 1 (0.09 = 9%).If we see values like 9 or 20, it means discount is in percent and we’ll convert.
# There are some negative discounts, this will be illogical. We will need to address this in data cleaning.

count    5000.000000
mean        0.063534
std         0.118311
min         0.000000
25%         0.000000
50%         0.000000
75%         0.090000
max         0.500000
Name: Discount, dtype: float64

## Validate Revenue Calculation
Since we already have a Revenue column, we must check if it’s consistent.

In [6]:
df['Revenue_calc']= df['Quantity'] * df['Price'] * (1 - df['Discount'])
(df['Revenue'] - df['Revenue_calc']).abs().sum()
# The total difference between existing Revenue and calculated Revenue is 0. 
# This means the existing Revenue column is accurate and consistent with other columns. 
# We will keep the existing Revenue column and decide on the calculated one if any row is higher.
#df.head()

np.float64(4.728928360009377e-11)

## Duplicates check (Order_ID)
confirm with evidence if Order_ID duplicates exist.

In [7]:
df.duplicated().sum(),
df['Order_ID'].duplicated().sum()

np.int64(0)

## Logical rule checks

In [8]:
{
    "qty_le_0": (df["Quantity"] <= 0).sum(),
    "price_lt_0": (df["Price"] < 0).sum(),
    "discount_lt_0": (df["Discount"] < 0).sum(),
    "discount_gt_1": (df["Discount"] > 1).sum(),
    "revenue_lt_0": (df["Revenue"] < 0).sum(),
}
# There are no rows with Quantity less than or equal to 0, Price less than 0, 
# Discount less than 0 or greater than 1, and Revenue less than 0. 
# These will not need to be addressed in data cleaning.

{'qty_le_0': np.int64(0),
 'price_lt_0': np.int64(0),
 'discount_lt_0': np.int64(0),
 'discount_gt_1': np.int64(0),
 'revenue_lt_0': np.int64(0)}

## Cleaning Summary

- Order_Date was successfully converted to datetime with no parsing errors.
- Discount values are stored as decimals between 0 and 1 and required no adjustment.
- Revenue values were validated against Quantity, Price, and Discount and found to be consistent.
- No duplicate rows or duplicate Order_ID values were identified.
- No logical data quality issues (negative or zero values) were detected.

As a result, no rows were removed. The dataset is considered clean and analysis-ready.


## Export the clean dataset

In [9]:
df.to_csv("outputs/globex_retail_clean.csv", index=False)
